In [1]:
# Libs
import pandas as pd
import os 
import json


In [2]:
pwd = os.getcwd()
# Dataset
df = pd.read_csv(pwd + r"\Data\Data - Aircraft_Incident_Dataset.csv")
dataset = pd.read_csv(pwd + r"\Data\Data - Jadaero Parts Cleaned.csv")
client_dataset = pd.read_csv(pwd + r"\Data\Data - Clients.csv")
dataset = dataset.copy()
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PartNumber   500 non-null    object
 1   PartName     500 non-null    object
 2   Description  495 non-null    object
dtypes: object(3)
memory usage: 11.8+ KB


In [3]:
IC = df['Incident_Cause(es)'].unique().tolist()
IC

# IMPORTING CATEGORIES AND GIVE IT TO DEEPSEEK TO CREATE A DICTIONARY

['Airplane - Engines, Airplane - Engines - Prop/turbine blade separation, Collision - Object, Collision - Object - Bird, Result - Emergency, forced landing - On runway',
 'Airplane - Undercarriage, Airplane - Undercarriage - Landing gear collapse, Result - Runway mishap',
 'Cargo - Fire/smoke, Result - Damaged on the ground',
 'Result - Damaged on the ground',
 'Landing/takeoff - Tailstrike, Result - Runway mishap',
 'Result - Runway excursion',
 'Info-Unavailable',
 'Result - Runway mishap',
 'Result - Loss of control',
 'Result - Loss of control, Result - Loss of control (presumed)',
 'Landing/takeoff - Landing, Landing/takeoff - Landing - Heavy, Result - Runway mishap',
 'Result - Emergency, forced landing - Ditching',
 'Landing/takeoff - Takeoff - Aborted, Result - Runway excursion',
 'Landing/takeoff - Landing, Landing/takeoff - Landing - Wrong runway/taxiway, Result - Runway excursion',
 'Airplane - Engines, Airplane - Engines - All engine powerloss, Result - Emergency, forced la

In [4]:
# Load JSON file
with open(pwd + '\Dics\Dic1 - Airsystems.json', 'r') as file:
    aircraft_systems = json.load(file)

# Match part numbers and descriptions to categories
def categorize_part(part_name, description):
    description = str(description).lower()
    part_name = str(part_name).lower()
    
    for category, data in aircraft_systems.items():
        # Check if part name matches known patterns
        if any(common_part.lower() in part_name for common_part in data['common_parts']):
            return category
        # Check description keywords
        if any(keyword.lower() in description for keyword in data['keywords']):
            return category
        # Check part name keywords too
        if any(keyword.lower() in part_name for keyword in data['keywords']):
            return category
    return 'other'

def get_risk_level(part_name, description):
    # Get risk level for aircraft parts based on categorization"""
    category = categorize_part(part_name, description)
    
    if category in aircraft_systems:
        return aircraft_systems[category]['risk_level']
    else:
        return 'Unknown'

def get_primary_replacement_interval(part_name, description):
    # Get the most relevant replacement interval for a specific part"""
    category = categorize_part(part_name, description)
    part_name_upper = str(part_name).upper()
    
    if category in aircraft_systems:
        replacement_dict = aircraft_systems[category].get('replacement_interval', {})
        
        # Try to find the most specific match first
        for component, interval in replacement_dict.items():
            if any(word in part_name_upper for word in component.upper().split()):
                return interval
        
        # If no specific match, return the first available interval
        if replacement_dict:
            return list(replacement_dict.values())[0]
    
    return 'On-condition'

def get_boeing_targets(part_name, description):
    # Get Boeing targets for the part category"""
    category = categorize_part(part_name, description)
    if category in aircraft_systems:
        return ', '.join(aircraft_systems[category].get('boeing_targets', []))
    return ''

def get_bombardier_targets(part_name, description):
    # Get Bombardier targets for the part category"""
    category = categorize_part(part_name, description)
    if category in aircraft_systems:
        return ', '.join(aircraft_systems[category].get('bombardier_targets', []))
    return ''

def get_mro_targets(part_name, description):
    # Get MRO targets for the part category"""
    category = categorize_part(part_name, description)
    if category in aircraft_systems:
        return ', '.join(aircraft_systems[category].get('mro_targets', []))
    return ''

def get_geo_targets(part_name, description):
    # Get geographical targets for the part category
    category = categorize_part(part_name, description)
    if category in aircraft_systems:
        return ', '.join(aircraft_systems[category].get('geo_targets', []))
    return ''

# Apply to your DataFrame
dataset['DamageCategory'] = dataset.apply(lambda x: categorize_part(x['PartName'], x['Description']), axis=1)
dataset['risk_level'] = dataset.apply(lambda x: get_risk_level(x['PartName'], x['Description']), axis=1)
dataset['replacement_interval'] = dataset.apply(lambda x: get_primary_replacement_interval(x['PartName'], x['Description']), axis=1)
dataset['boeing_targets'] = dataset.apply(lambda x: get_boeing_targets(x['PartName'], x['Description']), axis=1)
dataset['bombardier_targets'] = dataset.apply(lambda x: get_bombardier_targets(x['PartName'], x['Description']), axis=1)
dataset['mro_targets'] = dataset.apply(lambda x: get_mro_targets(x['PartName'], x['Description']), axis=1)
dataset['geo_targets'] = dataset.apply(lambda x: get_geo_targets(x['PartName'], x['Description']), axis=1)

# View results
dataset[dataset['DamageCategory'] != 'other'].sample(10)

,PartNumber,PartName,Description,DamageCategory,risk_level,replacement_interval,boeing_targets,bombardier_targets,mro_targets,geo_targets
322,GG433-1005-7,FAN AIR VALVE,Part Number: GG433-1005-7\r\rDescription: VALV...,fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...
239,65-90305-67,FILTER,Part Number: 65-90305-67\r\rDescription: FILTE...,fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...
13,338-067-014-0,FAN OUTLET GUIDE VANE,Part Number: 338-067-014-0\r\rDescription: VAN...,engines,Extreme,On-condition (ultrasound every 500 cycles),"CFM56, GE90, Trent, PW4000, LEAP","PW100, CF34, BR700","Engine overhaul, Hot section repair, Component...","Harsh environments (desert, coastal) + long-ha..."
267,F70199-1,TIRE GAUGE,Part Number: F70199-1\r\rDescription: TYRE PRE...,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid..."
53,PR1425B-1/2-CF,SEALANT,Part Number: PR1425B-1/2-CF\r\rDescription: SE...,engines,Extreme,On-condition (ultrasound every 500 cycles),"CFM56, GE90, Trent, PW4000, LEAP","PW100, CF34, BR700","Engine overhaul, Hot section repair, Component...","Harsh environments (desert, coastal) + long-ha..."
292,446055,WIRE CONNECTOR,Part Number: 446055\r\rDescription: WIRE CONNE...,electrical_systems,High,15 years (or insulation < 50MÎ©),"B737 electrical, B777 electrical, B787 electrical","CRJ electrical, Global electrical","Electrical component repair, Wire harness repl...",Older fleets needing upgrades + EM interferenc...
165,1001246-3,"VALVE, HIGH STAGE BLEED",Part Number: 1001246-3\r\rDescription: BLEED P...,fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...
459,3876287-1,MODULE DATA MEMORY,Part Number: 3876287-1\r\rDescription: MODULE-...,flight_management,Moderate,10 years (or MTBF),"B737 FMS, B777 FMS, B787 FMS","CRJ FMS, Global FMS","Avionics repair, Navigation calibration, Datab...","High-frequency routes (US domestic, Europe-Mid..."
54,4501089A,NOZZLE ASSY,Part Number: 4501089A\r\rDescription: NOZZLE A...,engines,Extreme,On-condition (ultrasound every 500 cycles),"CFM56, GE90, Trent, PW4000, LEAP","PW100, CF34, BR700","Engine overhaul, Hot section repair, Component...","Harsh environments (desert, coastal) + long-ha..."
433,2LA005163-25,LIGHT ASSY MAP,Part Number: 2LA005163-25\r\rDescription: LIGH...,cabin_systems,Low,On-condition,"B737 cabin, B777 cabin, B787 cabin","CRJ cabin, Global cabin","Cabin refurbishment, IFE upgrade, Seat repair",High-passenger-turnover routes + wear-intensiv...


In [5]:
dataset = dataset.rename(columns={'damage_category': 'DamageCategory'})
dataset[dataset['DamageCategory'] != 'other'].sample(10)

#RENAME COLUMN DamageCategory

,PartNumber,PartName,Description,DamageCategory,risk_level,replacement_interval,boeing_targets,bombardier_targets,mro_targets,geo_targets
278,03-5800-0010,STABILIZER,Part Number: 03-5800-0010\r\rDescription: SUPP...,flight_controls,Extreme,"5,000hrs (leak > 3 drops/hr)","B737 FCC, B777 PFCS, B787 FBW","CRJ FCS, Global FBW","Flight control rigging, Actuator overhaul, Com...","High-frequency routes (US domestic, Europe-Mid..."
240,320548-2,VALVE SOLENOID,"Part Number: 320548-2\r\rDescription: VALVE, T...",fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...
286,473597-31,M2K SMOKE DETECTOR,Part Number: 473597-31\r\rDescription: M2K SMO...,cargo_systems,Moderate,10 years (sensitivity test),"B747F cargo, B777F cargo, B767F cargo",DHC-8 cargo,"Cargo system maintenance, Door adjustment, Fir...",Hot climates + cargo temperature-sensitive ope...
404,1151968-1,CONTACTOR,Part Number: 1151968-1\r\rDescription: CONTACT...,electrical_systems,High,2 years (or capacity < 80%),"B737 electrical, B777 electrical, B787 electrical","CRJ electrical, Global electrical","Electrical component repair, Wire harness repl...",Older fleets needing upgrades + EM interferenc...
13,338-067-014-0,FAN OUTLET GUIDE VANE,Part Number: 338-067-014-0\r\rDescription: VAN...,engines,Extreme,On-condition (ultrasound every 500 cycles),"CFM56, GE90, Trent, PW4000, LEAP","PW100, CF34, BR700","Engine overhaul, Hot section repair, Component...","Harsh environments (desert, coastal) + long-ha..."
48,600-10910-5,GASKET WING TANK DOOR,Part Number: 600-10910-5\r\rDescription: GASKE...,fuel_systems,High,10 years (or weeping),"B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...
306,4100837G,BLEED CONTROL VALVE,Part Number: 4100837G\r\rDescription: VALVE AS...,fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...
101,144-186-000-011,VIBRATION SENSOR,Part Number: 144-186-000-011\r\rDescription: S...,engines,Extreme,On-condition (ultrasound every 500 cycles),"CFM56, GE90, Trent, PW4000, LEAP","PW100, CF34, BR700","Engine overhaul, Hot section repair, Component...","Harsh environments (desert, coastal) + long-ha..."
287,475571-29,DISCRETE CABIN SMOKE DETECTOR,Part Number: 475571-29\r\rDescription: DISCRET...,cargo_systems,Moderate,10 years (sensitivity test),"B747F cargo, B777F cargo, B767F cargo",DHC-8 cargo,"Cargo system maintenance, Door adjustment, Fir...",Hot climates + cargo temperature-sensitive ope...
102,60-755100-5,FUEL BOOSTER PUMP,Part Number: 60-755100-5\r\rDescription: MOTOR...,fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...


In [6]:
dataset[dataset['PartName'] == 'ENGINE NEW 675 HP']


,PartNumber,PartName,Description,DamageCategory,risk_level,replacement_interval,boeing_targets,bombardier_targets,mro_targets,geo_targets
499,PT6A-114A,ENGINE NEW 675 HP,NaN,other,Unknown,On-condition,,,,


In [7]:
dataset['DamageCategory'].value_counts()

# SHOW THE UNIQUE VALUES IN THE DamageCategory COLUMN
#  27 ROWS FROM 500 ROWS
# AFTER CLEANING AND SERCHING FOR OTHER PARTS 
#  253 ROWS FROM 500 ROWS

DamageCategory
other                         274
fuel_systems                   56
engines                        32
flight_controls                21
landing_gear                   20
cabin_systems                  18
flight_management              18
hydraulic_systems              15
electrical_systems             10
cargo_systems                   8
air_conditioning                7
surveillance_systems            6
pneumatic_systems               6
regional_aircraft_specific      3
communication_systems           3
ground_support                  3
Name: count, dtype: int64

In [8]:
dataset['Description'] = dataset['Description'].str.split('Description:', n=1,expand=True)[1]  # split the Description column by '\r\r' and keep the second part
dataset.sample(2)

,PartNumber,PartName,Description,DamageCategory,risk_level,replacement_interval,boeing_targets,bombardier_targets,mro_targets,geo_targets
435,GM428-4034-1,LINE ASSY FUEL,LINE ASSY FUEL,fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...
132,1152464-265,START POWER UNIT,APU START POWER UNIT,other,Unknown,On-condition,,,,


In [9]:
dataset.columns

Index(['PartNumber', 'PartName', 'Description', 'DamageCategory', 'risk_level',
       'replacement_interval', 'boeing_targets', 'bombardier_targets',
       'mro_targets', 'geo_targets'],
      dtype='object')

In [10]:
dataset = dataset.rename(columns={'PartNumber':'Part Number','PartName': 'Part Name', 'Description': 'Description', 'DamageCategory': 'Air System Category', 'risk_level': 'Risk Level',
                                  'replacement_interval': 'Replacement Interval', 'boeing_targets': 'Boeing Targets', 'bombardier_targets': 'Bombardier Targets',
                                  'mro_targets': 'MRO Targets', 'geo_targets': 'Geographical Targets'})

In [11]:
dataset.sample(1)

,Part Number,Part Name,Description,Air System Category,Risk Level,Replacement Interval,Boeing Targets,Bombardier Targets,MRO Targets,Geographical Targets
284,473227-01,FLAME DETECTOR,FLAME DETECTOR,other,Unknown,On-condition,,,,


In [12]:
description_null = dataset[dataset['Description'].isnull()]
description_null

,Part Number,Part Name,Description,Air System Category,Risk Level,Replacement Interval,Boeing Targets,Bombardier Targets,MRO Targets,Geographical Targets
0,021-355-0,"TIRE 11.00-12 10PLY 160 mph, 10 Ply",NaN,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid..."
1,M20101-01,TIRE MAIN,NaN,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid..."
12,1159SCL404-5,BRAKE ASSEMBLY,None,other,Unknown,On-condition,,,,
464,3876046-4,SOLENOID VALVE,NaN,fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...
479,472420-2,FIRE EXTINGUISHER,NaN,other,Unknown,On-condition,,,,
499,PT6A-114A,ENGINE NEW 675 HP,NaN,other,Unknown,On-condition,,,,


In [13]:
dataset['Description'] = dataset['Description'].fillna(dataset['Part Name'])
dataset[dataset['Part Name'] == 'TIRE MAIN']

,Part Number,Part Name,Description,Air System Category,Risk Level,Replacement Interval,Boeing Targets,Bombardier Targets,MRO Targets,Geographical Targets
1,M20101-01,TIRE MAIN,TIRE MAIN,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid..."


In [14]:
geo_keywords = []

for geo_target in dataset['Geographical Targets']:
    geo_target = str(geo_target).lower()
    locations = set()  # Use a set to avoid duplicates
    
    if 'us domestic' in geo_target:
        locations.add('United States')
    if 'europe' in geo_target:
        locations.add('Europe')
    if 'middle east' in geo_target:
        locations.add('Middle East')
    if 'desert' in geo_target:
        # Map deserts to countries/regions where deserts are located
        locations.update([
            'Morocco',        # Sahara Desert (Africa)
            'Mexico',         # Sonoran Desert (USA/Mexico)
            'United States',  # Sonoran Desert (USA/Mexico)
            'China',          # Gobi Desert (China)
            'Saudi Arabia'    # Arabian Desert (Middle East)
        ])
    if 'coastal' in geo_target:
        # Coastal regions with relevant countries
        locations.update([
            'Morocco',       # Casablanca
            'South Africa',  # Cape Town
            'United States', # Miami
            'Australia'      # Sydney
        ])
    if 'long-haul' in geo_target:
        # Long-haul routes — just tag as broad regions
        locations.update(['Europe', 'Asia', 'Morocco', 'United States'])

    if not locations:
        locations.add('Unknown')

    geo_keywords.append(list(locations))

dataset['Geographical Location'] = geo_keywords


In [15]:
dataset.sample(2)

,Part Number,Part Name,Description,Air System Category,Risk Level,Replacement Interval,Boeing Targets,Bombardier Targets,MRO Targets,Geographical Targets,Geographical Location
260,NSA931432A4BA0A,SWITCH,SWITCH,other,Unknown,On-condition,,,,,[Unknown]
88,338-067-007-0,VANE- OGV,VANE-OUTLET GUIDE DOUBLET,engines,Extreme,On-condition (ultrasound every 500 cycles),"CFM56, GE90, Trent, PW4000, LEAP","PW100, CF34, BR700","Engine overhaul, Hot section repair, Component...","Harsh environments (desert, coastal) + long-ha...","[Saudi Arabia, Asia, Mexico, Morocco, South Af..."


In [16]:


# Client Type 1: Regional Airlines
# Expanded geo pattern for Regional Airlines
geo1_pattern = r"(?i)high-frequency routes|regional routes|high-cycle operations"

clt1_mask = (
    dataset['Replacement Interval'].isin(['10 years', '15 years', '7 years']) |
    dataset['Geographical Targets'].str.contains('High-frequency routes') |
    dataset['Bombardier Targets'].str.contains('Regional aircraft') |
    dataset['Boeing Targets'].str.contains('Regional aircraft')
)



# Client Type 2: Cargo Airlines
Geo = 'Harsh environments|(desert, coastal)|long-haul operations|Hot climates|cargo temperature-sensitive operations'
clt2_mask = (
        dataset['Replacement Interval'].isin(['5,000hrs']) |
        dataset['Geographical Targets'].str.contains(Geo) |
        dataset['MRO Targets'].str.contains('Heavy maintenance')
)



# Client Type 3: MRO Service Providers
Geo2 = 'Safety-critical universal demand|harsh environments'
clt3_mask = (
        dataset['Replacement Interval'].isin(['6 months', '2,000hrs', '500 cycles']) |
        dataset['Geographical Targets'].str.contains(Geo2) |
        dataset['MRO Targets'].str.contains('Primary focus')
)


# Client Type 4: Legacy Fleet Operators
Geo3 = 'Older fleets needing upgrades|EM interference environments'
clt4_mask = (
        dataset['Geographical Targets'].str.contains(Geo3) |
        dataset['Boeing Targets'].str.contains('Older Boeing models')
)

# Step 1: Create a new column with default value (e.g. 0 for "Unclassified")
dataset['Client Number'] = 0
dataset.loc[clt1_mask, 'Client Number'] = 1
dataset.loc[clt2_mask, 'Client Number'] = 2
dataset.loc[clt3_mask, 'Client Number'] = 3
dataset.loc[clt4_mask, 'Client Number'] = 4

#Use (?i) for inline case-insensitive regex.


C:\Users\The Dev Reda\AppData\Local\Temp\ipykernel_4272\3508876311.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dataset['Geographical Targets'].str.contains(Geo) |


In [17]:
dataset

,Part Number,Part Name,Description,Air System Category,Risk Level,Replacement Interval,Boeing Targets,Bombardier Targets,MRO Targets,Geographical Targets,Geographical Location,Client Number
0,021-355-0,"TIRE 11.00-12 10PLY 160 mph, 10 Ply","TIRE 11.00-12 10PLY 160 mph, 10 Ply",landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...","[Saudi Arabia, Asia, Mexico, Morocco, Middle E...",3
1,M20101-01,TIRE MAIN,TIRE MAIN,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...","[Saudi Arabia, Asia, Mexico, Morocco, Middle E...",3
2,M01103-02,TIRE – MAIN,46x17.0R20 30PR 225mph Radial TL Michelin Air…,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...","[Saudi Arabia, Asia, Mexico, Morocco, Middle E...",3
3,114079019,"VALVE, LANDING GEAR SELECTOR",ELECTRO-HIDRAULIC SELECTORS,landing_gear,Extreme,"15,000 cycles (or structural cracks)","B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...","[Saudi Arabia, Asia, Mexico, Morocco, Middle E...",3
4,89794077,CYLINDER & VALVE ASSY,PORTABLE OXYGEN CYLINDER AND VALVE ASSEMBLY,fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...,"[Asia, Europe, United States, Morocco]",2
...,...,...,...,...,...,...,...,...,...,...,...,...
495,424634,CARTRIDGE 20PC,CARTRIDGE 20PC PACKAGE KIT 1.4C AS 1.4S,other,Unknown,On-condition,,,,,[Unknown],0
496,G825100-4,FIREX,"FIREX, DISPOSABLE (POTTY)",other,Unknown,On-condition,,,,,[Unknown],0
497,475089-1,FIREX 224 CU,FIREX 224 CU IN DUAL OUTLET,other,Unknown,On-condition,,,,,[Unknown],0
498,474158-3,FIREX,FIREX DUAL OUTLET 1400 CU IN,other,Unknown,On-condition,,,,,[Unknown],0


In [18]:
dataset_merged = dataset.merge(client_dataset, on='Client Number', how='left')
dataset_merged

,Part Number,Part Name,Description,Air System Category,Risk Level,Replacement Interval,Boeing Targets,Bombardier Targets,MRO Targets,Geographical Targets,Geographical Location,Client Number,Client Role
0,021-355-0,"TIRE 11.00-12 10PLY 160 mph, 10 Ply","TIRE 11.00-12 10PLY 160 mph, 10 Ply",landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...","[Saudi Arabia, Asia, Mexico, Morocco, Middle E...",3,MRO Service Providers
1,M20101-01,TIRE MAIN,TIRE MAIN,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...","[Saudi Arabia, Asia, Mexico, Morocco, Middle E...",3,MRO Service Providers
2,M01103-02,TIRE – MAIN,46x17.0R20 30PR 225mph Radial TL Michelin Air…,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...","[Saudi Arabia, Asia, Mexico, Morocco, Middle E...",3,MRO Service Providers
3,114079019,"VALVE, LANDING GEAR SELECTOR",ELECTRO-HIDRAULIC SELECTORS,landing_gear,Extreme,"15,000 cycles (or structural cracks)","B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...","[Saudi Arabia, Asia, Mexico, Morocco, Middle E...",3,MRO Service Providers
4,89794077,CYLINDER & VALVE ASSY,PORTABLE OXYGEN CYLINDER AND VALVE ASSEMBLY,fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...,"[Asia, Europe, United States, Morocco]",2,Cargo Airlines
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,424634,CARTRIDGE 20PC,CARTRIDGE 20PC PACKAGE KIT 1.4C AS 1.4S,other,Unknown,On-condition,,,,,[Unknown],0,Unclassified
496,G825100-4,FIREX,"FIREX, DISPOSABLE (POTTY)",other,Unknown,On-condition,,,,,[Unknown],0,Unclassified
497,475089-1,FIREX 224 CU,FIREX 224 CU IN DUAL OUTLET,other,Unknown,On-condition,,,,,[Unknown],0,Unclassified
498,474158-3,FIREX,FIREX DUAL OUTLET 1400 CU IN,other,Unknown,On-condition,,,,,[Unknown],0,Unclassified


In [19]:
dataset_merged = dataset_merged.explode('Geographical Location')
dataset_merged


,Part Number,Part Name,Description,Air System Category,Risk Level,Replacement Interval,Boeing Targets,Bombardier Targets,MRO Targets,Geographical Targets,Geographical Location,Client Number,Client Role
0,021-355-0,"TIRE 11.00-12 10PLY 160 mph, 10 Ply","TIRE 11.00-12 10PLY 160 mph, 10 Ply",landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...",Saudi Arabia,3,MRO Service Providers
0,021-355-0,"TIRE 11.00-12 10PLY 160 mph, 10 Ply","TIRE 11.00-12 10PLY 160 mph, 10 Ply",landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...",Asia,3,MRO Service Providers
0,021-355-0,"TIRE 11.00-12 10PLY 160 mph, 10 Ply","TIRE 11.00-12 10PLY 160 mph, 10 Ply",landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...",Mexico,3,MRO Service Providers
0,021-355-0,"TIRE 11.00-12 10PLY 160 mph, 10 Ply","TIRE 11.00-12 10PLY 160 mph, 10 Ply",landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...",Morocco,3,MRO Service Providers
0,021-355-0,"TIRE 11.00-12 10PLY 160 mph, 10 Ply","TIRE 11.00-12 10PLY 160 mph, 10 Ply",landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...",Middle East,3,MRO Service Providers
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,424634,CARTRIDGE 20PC,CARTRIDGE 20PC PACKAGE KIT 1.4C AS 1.4S,other,Unknown,On-condition,,,,,Unknown,0,Unclassified
496,G825100-4,FIREX,"FIREX, DISPOSABLE (POTTY)",other,Unknown,On-condition,,,,,Unknown,0,Unclassified
497,475089-1,FIREX 224 CU,FIREX 224 CU IN DUAL OUTLET,other,Unknown,On-condition,,,,,Unknown,0,Unclassified
498,474158-3,FIREX,FIREX DUAL OUTLET 1400 CU IN,other,Unknown,On-condition,,,,,Unknown,0,Unclassified


In [20]:
dataset['Geographical Targets'].unique()

array(['High-frequency routes (US domestic, Europe-Middle East), Harsh environments (desert, coastal) + long-haul operations, High-cycle operations + harsh environments',
       'Hot climates + cargo temperature-sensitive operations, High-altitude routes + extreme temperature operations, Long-haul operations',
       '',
       'High-passenger-turnover routes + wear-intensive operations, Long-haul operations + passenger comfort, High-frequency routes (US domestic, Europe-Middle East)',
       'Harsh environments (desert, coastal) + long-haul operations, High-altitude routes + extreme temperature operations, Weather-intensive routes',
       'Harsh environments (desert, coastal) + long-haul operations, High-altitude routes + extreme temperature operations, Safety-critical universal demand + harsh environments',
       'High-frequency routes (US domestic, Europe-Middle East), High-cycle operations + temperature extremes, Harsh environments (desert, coastal) + long-haul operations',
     

In [21]:
dataset_merged['Client Role'].value_counts()

Client Role
Cargo Airlines            612
MRO Service Providers     560
Unclassified              274
Legacy Fleet Operators    115
Regional Airlines          90
Name: count, dtype: int64

In [22]:
dataset_merged[dataset_merged['Replacement Interval'] == '12 years (rubber degradation)']

,Part Number,Part Name,Description,Air System Category,Risk Level,Replacement Interval,Boeing Targets,Bombardier Targets,MRO Targets,Geographical Targets,Geographical Location,Client Number,Client Role
395,MF10-08-01,OXYGEN MASK,"OXYGEN MASK, CREW",cabin_systems,Low,12 years (rubber degradation),"B737 cabin, B777 cabin, B787 cabin","CRJ cabin, Global cabin","Cabin refurbishment, IFE upgrade, Seat repair",High-passenger-turnover routes + wear-intensiv...,Asia,1,Regional Airlines
395,MF10-08-01,OXYGEN MASK,"OXYGEN MASK, CREW",cabin_systems,Low,12 years (rubber degradation),"B737 cabin, B777 cabin, B787 cabin","CRJ cabin, Global cabin","Cabin refurbishment, IFE upgrade, Seat repair",High-passenger-turnover routes + wear-intensiv...,Morocco,1,Regional Airlines
395,MF10-08-01,OXYGEN MASK,"OXYGEN MASK, CREW",cabin_systems,Low,12 years (rubber degradation),"B737 cabin, B777 cabin, B787 cabin","CRJ cabin, Global cabin","Cabin refurbishment, IFE upgrade, Seat repair",High-passenger-turnover routes + wear-intensiv...,Middle East,1,Regional Airlines
395,MF10-08-01,OXYGEN MASK,"OXYGEN MASK, CREW",cabin_systems,Low,12 years (rubber degradation),"B737 cabin, B777 cabin, B787 cabin","CRJ cabin, Global cabin","Cabin refurbishment, IFE upgrade, Seat repair",High-passenger-turnover routes + wear-intensiv...,United States,1,Regional Airlines
395,MF10-08-01,OXYGEN MASK,"OXYGEN MASK, CREW",cabin_systems,Low,12 years (rubber degradation),"B737 cabin, B777 cabin, B787 cabin","CRJ cabin, Global cabin","Cabin refurbishment, IFE upgrade, Seat repair",High-passenger-turnover routes + wear-intensiv...,Europe,1,Regional Airlines


In [23]:
# print(dataset_merged['Geographical Location'].unique()) #Unknown
# print(dataset_merged['Risk Level'].unique()) #Unknown
# print(dataset_merged['Air System Category'].unique()) #other
# print(dataset_merged['Replacement Interval'].unique()) # On-condition
# print(dataset_merged['Client Role'].unique())  #'Unclassified

dataset_filtred = dataset_merged[
    (dataset_merged['Geographical Location'] != 'Unknown') &
    (dataset_merged['Risk Level'] != 'Unknown') & 
    (dataset_merged['Air System Category'] != 'other') &
    (dataset_merged['Replacement Interval'] != 'On-condition') &
    (dataset_merged['Client Role'] != 'Unclassified')
]

In [24]:
dataset_filtred.to_csv(pwd + r"\Data\Data - Jadaero Parts Edited.csv", index=False)
# Save the categorized DataFrame to a new CSV file